In [33]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q openapi
!pip install -q tqdm
!pip install -q openai


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGBot/0.0' ,'en')
doc = wiki.page("Hayao_Miyazaki").text
paragraphs = doc.split('\n\n')

In [ ]:
import textwrap
for i,p in enumerate(paragraphs):
    wrapped_text = textwrap.fill(p, width=100)
    print("-=================------")
    print(wrapped_text)
    print("-=================------")

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)
docs_embed = model.encode(paragraphs, normalize_embeddings=True)

In [186]:
import numpy as np
query = "What schools did Miyazaki gratuate?"
query_embed = model.encode(query, normalize_embeddings=True)
similarities = np.dot(docs_embed, query_embed.T)
top_5_idx = np.argsort(similarities, axis=0)[-5:][::-1].tolist()
most_similar_documents = [paragraphs[i] for i in top_5_idx]

In [187]:
CONTEXT = ""
for i , p in enumerate(most_similar_documents):
    CONTEXT += textwrap.fill(p, width=100) + "\n\n"

prompt = f"""
use the following context CONTEXT to answer the QUESTION at the end. Be succint:

CONTEXT: {CONTEXT}
QUESTION: {query}
"""
from openai import OpenAI
# client = OpenAI(
#     base_url = 'http://localhost:11434/v1',
#     api_key='ollama', # required, but unused
# )
client = OpenAI(
    api_key='...',
)
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt}
  ]
)
print(response.choices[0].message.content)

Hayao Miyazaki graduated from Eifuku Elementary School and Ōmiya Junior High School. He then graduated from Toyotama High School and Gakushuin University, where he earned degrees in political science and economics.


In [188]:
top_5_idx

[2, 0, 17, 15, 7]